In [16]:
from queue import Queue
from math import pi

In [17]:
class Graph:
    def __init__(self, adj_list):
        self.adj_list = adj_list
        self.reverse = {}
        self.edges = []
        self.nodes = set()
        self.__dummy_count__ = 0

        for v in adj_list.keys():
            self.nodes.add(v)
            for u in adj_list[v]:
                self.nodes.add(u)
                self.edges.append((v,u))
                self.reverse.setdefault(u,[]).append(v)

    def neighbors(self, node):
        return set(self.adj_list.setdefault(node, []) + self.reverse.setdefault(node, []))
    
    def add_dummy_node(self):
        new_node = f'#DUMMY_NODE_{self.__dummy_count__}#'
        self.__dummy_count__ += 1
        self.nodes.add(new_node)
        return new_node
    
    def add_edge(self, edge: tuple, directed: bool):
        v = edge[0]
        u = edge[1]
        if v not in self.nodes or u not in self.nodes:
            raise Exception(f'Edge ({v, u}) has one endpoint, which doesnt belong to a set of vertices')
        
        if directed:
            self.edges.append((v, u))
            self.adj_list.setdefault(v, []).append(u)
            self.reverse.setdefault(u, []).append(v)
        else:
            self.edges.append((v, u))
            self.edges.append((u, v))
            self.adj_list.setdefault(u, []).append(v)
            self.adj_list.setdefault(v, []).append(u)
            self.reverse.setdefault(u, []).append(v)
            self.reverse.setdefault(v, []).append(u)

    def delete_edge(self, edge: tuple):
        '''
        Removes a directed edge. 
        '''
        def remove_edge(adj_list, edge):
            self.adj_list[edge[0]].remove(edge[1])

            if len(self.adj_list[edge[0]]) == 0:
                del adj_list[edge[0]]
            return adj_list


        v = edge[0]
        u = edge[1]
        if v not in self.nodes or u not in self.nodes:
            raise Exception(f'Edge ({v, u}) has one endpoint, which doesnt belong to a set of vertices')
        
        self.edges.remove(edge)

        self.adj_list = remove_edge(self.adj_list, (v, u))
        self.reverse = remove_edge(self.reverse, (u, v))

        # Neither references nor is referenced by anything
        if u not in self.adj_list.keys() and u not in self.reverse.keys():
            del self.nodes[u]

        if v not in self.adj_list.keys() and v not in self.reverse.keys():
            del self.nodes[v]

    def get_edges_starting(self, vertex):
        return set(filter(lambda e: e[0] == vertex))

    def get_edges(self, start, end):
        edges_w_start = set(filter(lambda e: e[0] == start or e[1] == start, self.edges))

        if start == end:
            return edges_w_start

        res = set()
        for edge in edges_w_start:
            if end == edge[0] or end == edge[1]:
                res.add(edge)

        return res

In [3]:
dag = DAG({1 : [2,3,4], 2 : [5,6], 3: [5]})
print(dag.neighbors(5))

{2, 3}


In [5]:
def k_core_decomposition(graph: Graph):
    '''
    Computes k-Core decomposition of the given DAG.

    Parameters
    ------
    graph : DAG
            directed acyclic graph to decompose
    
    Returns
    ------
    A dictionary, where keys are vertices and values are their core numbers.

    A k-core decomposition of a graph is a hierarchical decomposition, where a threshold k is set on a degree of
    each vertex, and those vertices which don't satisfy threshold are excluded from the process. Rinse and repeat - 
    start from lowest.

    Useful: degeneracy - highest k, such that H a subgraph of G is not empty.
    '''
    size = len(graph.nodes)

    max_deg = 0
    deg = [None]*size
    order = [None]*size
    number = {}

    for idx, node in enumerate(graph.nodes):
        deg[idx] = len(graph.neighbors(node))
        number[node] = idx
        order[idx] = node
        max_deg = max(max_deg, deg[idx])

    bin = [0]*(max_deg + 1)
    
    # Count vertices in each bin.
    for i in range(0, size):
        bin[deg[i]] += 1 

    # Sort by degree. Determine starting vertex in each bin.
    start = 0
    for i in range(0, max_deg + 1):
        tmp = bin[i]
        bin[i] = start
        start = start + tmp

    # Put sorted vertices into vert
    vert = [None]*size
    pos = [None]*size
    for i in range(0, size):
        pos[i] = bin[deg[i]]
        vert[pos[i]] = i
        bin[deg[i]] += 1

    # Recover starts.
    for i in reversed(range(1, max_deg + 1)):
        bin[i] = bin[i - 1]
    bin[0] = 0

    core = {}
    # Calculate core numbers.
    for i in range(0, size):
        curr_node = order[vert[i]]

        core[curr_node] = deg[vert[i]]

        for nb in graph.neighbors(curr_node):
            u_idx = number[nb]

            # Decrease vertex degrees.
            if deg[u_idx] > deg[vert[i]]:
                pu = pos[u_idx]
                pw = bin[deg[u_idx]]
                
                w_idx = vert[pw]
                if u_idx != w_idx:
                    pos[u_idx] = pw
                    pos[w_idx] = pu
                    vert[pu] = w_idx
                    vert[pw] = u_idx

                bin[deg[u_idx]] += 1
                deg[u_idx] -= 1

    return core


In [15]:
# K-Core decomposition tests.
# dag_k_3 = DAG({1: [2,3,4], 5:[2,3,4], 2: [3], 4: [3]})
# k_3 = k_core_decomposition(dag_k_3)
# k_3_expected = {1: 3, 2: 3, 4: 3, 5: 3, 3: 3}
# assert(k_3 == k_3_expected)

# dag_k_1_3 = DAG({1: [2, 3, 4, 6], 5:[2, 3, 4], 2: [3, 6, 7], 4: [3], 7: [8], 8: [9]})
# k_1_3 = k_core_decomposition(dag_k_1_3)
# k_1_3_expected = {9: 1, 8: 1, 7: 1, 6: 2, 4: 3, 5: 3, 1: 3, 2: 3, 3: 3}
# assert(k_1_3 == k_1_3_expected)

In [11]:
class HierarchyLayoutService:
    '''
    Layous out a DAG, using Sugiyama framework with twists.
    '''
    def __init__(self, graph: Graph, leveling: dict):
        self.graph = Graph(graph.adj_list)
        self.leveling = leveling
        self.dummies = []
        
    def get_max_level(self):
        return max(self.leveling.values())

    def create_order(self, graph: Graph, highest_in_center: bool = True):
        max_lvl = self.get_max_level()
        order_temp = [[]]*max_lvl # +1 ?

        # - TODO: get rid of this, why should we allow for empty levels
        for n in graph.nodes:
            if highest_in_center:
                order_temp[max_lvl - k_core_decomposition[n]].append(n)
            else:
                order_temp[k_core_decomposition[n]].append(n)

        order = []
        # Remove empty levels.
        for lvl in order_temp:
            if len(lvl) != 0:
                order.append(lvl)
        
        return order

    def get_node_level_assigments(order: list):
        res = {} 
        for idx, lvl in order:
            for n in lvl:
                res[n] = idx
        return res
    
    def make_proper(self):
        '''
        Insert dummy nodes, to make hierarchy proper.
        '''

        edges = self.graph.edges

        for edge in edges:
            source = edge[0]
            target = edge[1]

            source_lvl = self.leveling[source]
            target_lvl = self.leveling[target]

            if abs(target_lvl - source_lvl) <= 1:
                continue

            start = target_lvl
            end = source_lvl

            curr_node = target
            end_node = source

            if source_lvl < target_lvl:
                start = source_lvl
                end = target_lvl
                curr_node = source
                end_node = target

            for i in range(start+1, end):
                dummy = self.graph.add_dummy_node()
                self.order[i].append(dummy)
                self.leveling[dummy] = i

                self.dummies.append(dummy)

                self.graph.add_edge((curr_node, dummy), directed=False)

                curr_node = dummy

            # Add final
            self.graph.add_edge((curr_node, end_node), directed=False)

            # Delete old edge
            self.graph.delete_edge(edge)

In [12]:
class LayoutCrossingOptimizationService:
    def __init__(self, graph: Graph, order: list):
        '''
        Initializes the new instance of crossing reduction service.
        Parameters
        -------
        order : list
                A list, where each element is an array of nodes on that level.
        '''
        self.graph = graph
        self.order = order
        self.offset = {}

    def minimize_crossings(self):
        if len(self.offset) > 0:
            return self.offset
        return
    
    def get_vertical_edges(self, upper_lvl : list, lower_lvl : list):
        for v in 
    
    def minimize_edge_lengths(self):
        '''
        Minimizes edge lengths by rotating graph.
        '''

        for i in range(0, len(self.order) - 1):
            inner_angle_inc = 2*pi / len(self.order[i])
            outer_angle_inc = 2*pi / len(self.order[i+1])

            # Avg ange spanned by all edges
            avg_spanned_angle = 0
            outer_angle = 0
            edge_cnt = 0

            for u in self.order[i + 1]:
                for idx, v in enumerate(self.order[i]):
                    for egde in self.graph.get_edges(u, v):
                        edge_cnt += 1

                        inner_angle = idx * inner_angle_inc
                        avg_spanned_angle += (inner_angle - outer_angle) + (-self.offset[e] * 2*pi)
                
                outer_angle += outer_angle_inc

            avg_spanned_angle /= edge_cnt
            rotation = round(avg_spanned_angle / outer_angle_inc)

            clockwise = rotation >= 0
            self.rotate(i + 1, rotation, clockwise)

            if len(self.order[i + 1]) == 1:
                for u in self.order[i+1]:
                    for e in self.graph.get_edges_starting(u):
                        self.offset[e] = 0

    def rotate(self, level : int, rotation: int, clockwise: bool):
        '''
        Rotates the current level
        level     : int
                    level number to rotate
        rotation  : int
                    number of positions to rotate
        clockwise : bool
                    direction of rotation

        Alters order and offset properties.
        '''

        next_lvl_offset_delta = -1 if clockwise else 1
        prev_lvl_offset_delta = 1 if clockwise else -1
        for _ in range(0, rotation):
            v = None

            if clockwise:
                # Remove last, add first
                v = self.order[level].pop()
                self.order[level].insert(0, v)
            else:
                # Remove first, add last
                v = self.order[level].pop(0)
                self.order[level].append(v)

                if level + 1 < len(self.order):
                    for u in self.order[level+1]:
                        for edge in  self.graph.get_edges(u, v):
                            self.offset[edge] += next_lvl_offset_delta
                
                for w in self.order[level-1]:
                    for edge in self.graph.get_edges(v, w):
                        self.offset[edge] += prev_lvl_offset_delta